# gitHub

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import utils

import re
from os import path

from datetime import datetime
from dateutil.relativedelta import relativedelta

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Input data

In [2]:
# Enter inputs
current_FY_start =  '2017/4/1'# 今年度が始まったカレンダー年月日 'YYYY/M/D'  (例：2017/4/1)
report_date = '2017/8/31'     # レポートする最終日 'YYYY/M/D'　(例：2017/8/31　2017年8月の終わりまで)

num_yr_to_aggregate = 5       #　集計する年数。例：5 (過去5年)

In [3]:
# Working directory　読み込むファイルのあるディレクトリを記入して下さい
filePath='/Users/yuka/Documents/Python/gitHub/Option Sales/'
#filePath=r'C:\\Users\\ymatsukawa\\Documents\\Project\\OptionSales\\gitHub\\'



# Output files 結果を出力するディレクトリーを記入して下さい
#outputPath = 'C:\\Users\\ymatsukawa\\Documents\\Project\\OptionSales\\gitHub\\output\\'
outputPath = '/Users/yuka/Documents/Python/gitHub/Option Sales/output/'


# Enter the file name 読み込むファイル名を記入して下さい

# KDC data KDCデータ
#kdc_last10FY = 'Sales - Soft・Serv_Prod_FY2010-2016.xlsx' # KDC 過去10FY販売データ
kdc_thisFY = 'Sales FY2018_05032018.xlsx' # KDC 昨年度FY販売データ
kdc_lastFY = 'Sales FY2017_05032018.xlsx' # KDC 今年度FY販売データ

# TA data　TAデータ
taDict = dict()
taDict[0] = 'TA_Sales_Report_FY2017_gitHub.xlsx'
            
taDict[4] = 'TA_Sales_Report_FY201804_gitHub.xlsx' # TA 今年度FY 4月販売データ
taDict[5] = 'TA_Sales_Report_FY201805_gitHub.xlsx' # TA 今年度FY 5月販売データ
taDict[6] = 'TA_Sales_Report_FY201806_gitHub.xlsx' # TA 今年度FY 6月販売データ
taDict[7] = 'TA_Sales_Report_FY201807_gitHub.xlsx' # TA 今年度FY 7月販売データ
taDict[8] = 'TA_Sales_Report_FY201808_gitHub.xlsx' # TA 今年度FY 8月販売データ
taDict[9] = r''
taDict[10] = r''
taDict[11] = r''
taDict[12] = r''
taDict[1] = r''
taDict[2] = r''
taDict[3] = r''

ta_margin =  'TA傘下の各拠点のマージン率.xlsx' # TA マージン率

ta_ex_thisFY = 'Exchange Rate_FY2018.xlsx' # TA 今年度為替レート
ta_ex_lastFY = 'Exchange Rate_FY2017.xlsx' # TA 昨年度為替レート

In [4]:
current_FY_start = pd.Timestamp(current_FY_start)
last_FY_start = current_FY_start - relativedelta(years = 1)
report_date = pd.Timestamp(report_date)

In [5]:
path.abspath(filePath+kdc_lastFY)

'/Users/yuka/Documents/Python/gitHub/Option Sales/Sales FY2017_05032018.xlsx'

## KDC Data

In [6]:
# Read in KDC data files　KDCデータファイルの読み込み
#kdc1 = pd.read_excel(filePath + kdc_last10FY,  skiprows=3)

kdc2 = pd.read_excel(path.abspath(filePath + kdc_lastFY),  skiprows=3)
kdc3 = pd.read_excel(path.abspath(filePath + kdc_thisFY),  skiprows=3)
#kdc2 = pd.read_excel(filePath + kdc_lastFY,  skiprows=3)
#kdc3 = pd.read_excel(filePath + kdc_thisFY,  skiprows=3)

In [7]:
# Concatenate data & 行数確認
kdc_data = pd.concat([kdc2, kdc3])
#kdc_data = pd.concat([kdc1, kdc2, kdc3])

#print('file 1 (row, col): ', kdc1.shape)
print('file 2 (row, col): ', kdc2.shape)
print('file 3 (row, col): ', kdc3.shape)

print('Number of duplicates to be removed: ', kdc_data[kdc_data.duplicated()].shape[0])
kdc_data.drop_duplicates(inplace=True)
print('Total: (row, col)', kdc_data.shape)

file 2 (row, col):  (80217, 15)
file 3 (row, col):  (35308, 15)
Number of duplicates to be removed:  7520
Total: (row, col) (108005, 15)


In [8]:
kdc_data.head(2)

,Unnamed: 0,Sort Key,Company Code,Company Name,Actual Sales,Actual Currency,Plan Sales,Plan Currency,Ratio to Plan,3rd Party or Inter,Year / Month,Material Name,Company Group Name,Sales Quantity,Actual Sales (Ex)
0,NaN,1,1001,Alley Cats Pet Shop,373138.0,JPY,NaN,JPY,NaN,2,201604,Non target products,08. Pow Production-G1,33.0,373138.0
1,NaN,1,1001,Alley Cats Pet Shop,-68318.0,JPY,NaN,JPY,NaN,2,201604,Non target products,08. Pow Production-G1,15.0,-68318.0


In [9]:
# Select only necessary columns 必要なカラムだけに限定
kdc_data2 = kdc_data[['Year / Month','Company Code', 'Company Name', 'Company Group Name','3rd Party or Inter',  
                      'Material Name','Sales Quantity','Actual Sales','Plan Currency' ,'Actual Sales (Ex)' ]].copy()
kdc_data2.head(2)

,Year / Month,Company Code,Company Name,Company Group Name,3rd Party or Inter,Material Name,Sales Quantity,Actual Sales,Plan Currency,Actual Sales (Ex)
0,201604,1001,Alley Cats Pet Shop,08. Pow Production-G1,2,Non target products,33.0,373138.0,JPY,373138.0
1,201604,1001,Alley Cats Pet Shop,08. Pow Production-G1,2,Non target products,15.0,-68318.0,JPY,-68318.0


In [10]:
# Clean up KDC data KDCデータのクリーンアップ

# Delete rows if necessary columns are null 　計算に必要なセルが空白な記録は除く
print(kdc_data2.shape)

kdc_data2.dropna(how='all', inplace=True)  
kdc_data2 = kdc_data2[kdc_data2['Material Name'].notnull()]
kdc_data2 = kdc_data2[kdc_data2['Actual Sales'].notnull()]
kdc_data2 = kdc_data2[kdc_data2['Year / Month'].notnull()]
kdc_data2 = kdc_data2[kdc_data2['Company Group Name'].notnull()]

# Limit to 3rd party only　3rd partyのみに
kdc_data2['3rd Party or Inter']=kdc_data2['3rd Party or Inter'].astype(int)
kdc_data2 = kdc_data2[(kdc_data2['3rd Party or Inter'])==3]

# Remove "Production-G and Production-G2". Don't need them.　販社名が”*Production-”は除く
#groups = ['01.KDE-G', '04.KDA-G', '05.OCEANIA-G', '06.KDAS-G', '07.KDCN', '09.KDJ', '11.KTST']
#kdc_data2 = kdc_data2[kdc_data2['Company Group Name'].isin(groups)]
kdc_data2 = kdc_data2[~kdc_data2['Company Group Name'].str.contains('roduction')]


# Convert to year/month to int
kdc_data2['Year / Month'] = kdc_data2['Year / Month'].astype(int)
#data_kdc2['3rd Party or Inter'] = data_kdc2['3rd Party or Inter'].astype(int)

# Add 'ExRate' column
kdc_data2['ExRate'] = kdc_data2['Actual Sales (Ex)']/kdc_data2['Actual Sales']

# Reorder columns カラムの順番を分かり易く変更
cols = ['Year / Month','Company Code', 'Company Name', 'Company Group Name','3rd Party or Inter',  
        'Material Name','Sales Quantity','Actual Sales','Plan Currency','ExRate','Actual Sales (Ex)' ]

kdc_data2 = kdc_data2[cols]

print('Total: (row, col)', kdc_data2.shape)
kdc_data2.head(2)

(108005, 10)
Total: (row, col) (71664, 11)


,Year / Month,Company Code,Company Name,Company Group Name,3rd Party or Inter,Material Name,Sales Quantity,Actual Sales,Plan Currency,ExRate,Actual Sales (Ex)
7768,201604,1101,Amazin' Animals Pet Shop,09. Pow Japan,3,Non target products,4.0,21000.0,JPY,1.0,21000.0
7769,201604,1101,Amazin' Animals Pet Shop,09. Pow Japan,3,Non target products,4.0,5460.0,JPY,1.0,5460.0


## TA Data

In [11]:
# Read in this FY TA data (new format) TAデータの読み込み

ta_files_to_read = [taDict[k] for k in sorted(taDict.keys()) if len(taDict[k])>0]
#ta_files_to_read = ['TA_sales_report_June_activation_items_2017_07_10.xlsx']
ta_data =pd.DataFrame([])
for f in ta_files_to_read:
    temp = pd.read_excel(filePath + f)#, sheetname='sales details')
    print('Reading ' + f, temp.shape)
    # set column names because it changed in the August file
    temp.columns=['document number', 'customer no', 'Partner', 'booking date', 'Item No','KD Item No', 'description', 
                  'QTY', 'landed price','total landed price', 'amount piece', 'total amount', 'Date']
    ta_data = ta_data.append(temp, ignore_index = True)  
 
print(ta_data.shape)    

Reading TA_Sales_Report_FY2017_gitHub.xlsx (13078, 13)
Reading TA_Sales_Report_FY201804_gitHub.xlsx (939, 13)
Reading TA_Sales_Report_FY201805_gitHub.xlsx (358, 13)
Reading TA_Sales_Report_FY201806_gitHub.xlsx (445, 13)
Reading TA_Sales_Report_FY201807_gitHub.xlsx (333, 13)
Reading TA_Sales_Report_FY201808_gitHub.xlsx (372, 13)
(15525, 13)


In [12]:
ta_data.head(3)

,document number,customer no,Partner,booking date,Item No,KD Item No,description,QTY,landed price,total landed price,amount piece,total amount,Date
0,1101773280,32866,Tier GmbH,2016-04-01 00:00:00,654010094,NaN,Facial Massage Roller for Cats,1,NaN,NaN,47.93,47.93,2016/04
1,1101772220,999003215,TA Deutschland GmbH,2016-04-01 00:00:00,654010094,NaN,Facial Massage Roller for Cats,8,NaN,NaN,35.1275,281.02,2016/04
2,1101772991,999003215,TA Deutschland GmbH,2016-04-01 00:00:00,653010091,NaN,U-pet Carriers,1,NaN,NaN,133.34,133.34,2016/04


In [13]:
# Remove header cells and blank lines 必要ない行を削除
temp =  ta_data.loc[ta_data['document number'].notnull() & ~ta_data['document number'].isin(['document number'])].copy()
temp.dropna(how ='all', inplace=True)
ta_data2 = temp[temp['Partner'].notnull()]

# Change these to int. They are int in the 201708 file
ta_data2['document number']=pd.to_numeric(ta_data2['document number'], errors='coerce')
ta_data2['customer no']=pd.to_numeric(ta_data2['document number'], errors='coerce')
ta_data2['Item No']=pd.to_numeric(ta_data2['document number'], errors='coerce')

# drop dups from after 2017/05 data only
#ta_data2.drop_duplicates(inplace=True)
temp1 = ta_data2[ta_data2.Date<'2017/05'].copy()
temp2 = ta_data2[ta_data2.Date>='2017/05'].copy()
temp2.drop_duplicates(inplace = True)
ta_data2 = pd.concat([temp1, temp2])

print(ta_data2.shape)
ta_data2.head(2)

(15501, 13)


,document number,customer no,Partner,booking date,Item No,KD Item No,description,QTY,landed price,total landed price,amount piece,total amount,Date
0,1101773280,1101773280,Tier GmbH,2016-04-01 00:00:00,1101773280,NaN,Facial Massage Roller for Cats,1,NaN,NaN,47.93,47.93,2016/04
1,1101772220,1101772220,TA Deutschland GmbH,2016-04-01 00:00:00,1101772220,NaN,Facial Massage Roller for Cats,8,NaN,NaN,35.1275,281.02,2016/04


In [14]:
ta_data2.Partner.unique()

array(['Tier GmbH', 'TA Deutschland GmbH', 'Bär GmbH', 'Eichhörnchen GmbH',
       'Fuchs GmbH', 'Hase GmbH', 'Hund GmbH', 'Kalb GmbH',
       'Kaninchen GmbH', 'Katze GmbH', 'Kätzchen GmbH',
       'TA Österreich GmbH', 'Kuh GmbH', 'Löwe GmbH', 'Maus GmbH',
       'Pferd GmbH', 'Ratte GmbH', 'Schildkröte GmbH', 'Schlange GmbH',
       'Stier GmbH', 'Wolf GmbH', 'Wurm GmbH', 'Vogel GmbH', 'Hahn GmbH',
       'Henne GmbH', 'Adler GmbH', 'Küken GmbH', 'Ameise GmbH',
       'Biene GmbH', 'Fliege GmbH', 'Heuschrecke GmbH', 'Mott GmbH',
       'TA Ceska', 'Mücke GmbH', 'Schmetterling GmbH', 'Spinne GmbH',
       'Huhn GmbH', 'Scheune GmbH', 'Brücke GmbH', 'Hügel GmbH',
       'Berg GmbH', 'Strand GmbH', 'See GmbH', 'Fluss GmbH', 'Straße GmbH',
       'Bauernhof GmbH', 'Feld GmbH', 'Wald GmbH', 'Pflanze GmbH',
       'Stadt GmbH', 'See / das Meer GmbH', 'Bucht GmbH', 'Gebirge GmbH',
       'Bach GmbH', 'Himmel GmbH', 'Insel GmbH', 'Luft GmbH', 'Wiese GmbH',
       'Wüste GmbH', 'Teich GmbH',

In [15]:
#### Partner2 カラム　下の４社は"TA", その他は"Others" 
#### TA Triumph-Adler Ceska 
#### TA Triumph-Adler Deutschland GmbH 
#### TA Triumph-Adler Österreich GmbH 
#### TA Triumph-Adler Schweiz AG

In [16]:
def assign_partner2(st):
    if 'TA CESKA' in st.upper():
        return 'TA Ceska'
    elif 'TA DEUTSCHLAND GMBH' in st.upper():
        return 'TA Deutschland GmbH'
    elif 'TA ÖSTERREICH GMBH' in st.upper():
        return 'TA Österreich GmbH'
    elif 'TA SCHWEIZ AG' in st.upper():
        return 'TA Schweiz AG'
    else:
        return 'Others'

In [17]:
# # 関数定義　上記4社TAはそのまま、他は"Others"とする
# # Function definition - Keep the company name for 4 TA, rename other TA to 'Others' 
# def assign_partner2(st):
#     if 'TA TRIUMPH-ADLER CESKA' in st.upper():
#         return 'TA Triumph-Adler Ceska'
#     elif 'TA TRIUMPH-ADLER DEUTSCHLAND GMBH' in st.upper():
#         return 'TA Triumph-Adler Deutschland GmbH'
#     elif 'TA TRIUMPH-ADLER ÖSTERREICH GMBH' in st.upper():
#         return 'TA Triumph-Adler Österreich GmbH'
#     elif 'TA TRIUMPH-ADLER SCHWEIZ AG' in st.upper():
#         return 'TA Triumph-Adler Schweiz AG'
#     else:
#         return 'Others'

In [18]:
# 'Partner2'カラムを追加して、上の関数を適用

#ta_data2.loc[:, 'Partner2'] = data_ta2['Partner'].apply(lambda x: _ta_name(x))
ta_data2['Partner2'] = ta_data2['Partner'].apply(lambda x: assign_partner2(x))
ta_data2['Group'] ='TA'
ta_data2['Currency']='Euro'

In [19]:
# KDCデータと統一する準備 - カラム名とデータタイプをKDCと同じに
# Reorder columns
col_ta = ['Date', 'customer no', 'Partner', 'Partner2', 'Group','description', 'QTY', 'total amount', 'Currency']
ta_data2 = ta_data2[col_ta]

# Remove '/' from the Date column
ta_data2['Date'] = ta_data2['Date'].str.replace('/', '')

 # Rename the 'Date' column to 'Month' 
ta_data2.columns=['Month', 'customer no', 'Partner', 'Partner2', 'Group','description', 'QTY', 'total amount', 'Currency']

# Change dtype of columns to match KDC_data
ta_data2['Month'] = ta_data2['Month'].astype(int)
ta_data2['customer no'] = ta_data2['customer no'].astype(np.int64)
ta_data2['QTY'] = ta_data2['QTY'].astype(float)
ta_data2['total amount'] = ta_data2['total amount'].astype(float)

ta_data2.head(2)

,Month,customer no,Partner,Partner2,Group,description,QTY,total amount,Currency
0,201604,1101773280,Tier GmbH,Others,TA,Facial Massage Roller for Cats,1.0,47.93,Euro
1,201604,1101772220,TA Deutschland GmbH,TA Deutschland GmbH,TA,Facial Massage Roller for Cats,8.0,281.02,Euro


## TA Exange & Margin Rate   為替とマージン率

In [20]:
# 為替レートの読み込み
# Read in Fx Rate data
temp_fx_thisFY = pd.read_excel(filePath + ta_ex_thisFY, sheetname='Exchange Rate (Actual)', skiprows=3)
temp_fx_lastFY = pd.read_excel(filePath + ta_ex_lastFY, sheetname='Exchange Rate (Actual)', skiprows=3) #index_col='Currency(from)'
margin = pd.read_excel(filePath + ta_margin, sheetname='Sheet1', skiprows=1)  #

In [21]:
# 必要なカラムの選択、昨年度と今年度の為替データを１つに
fx_thisFY=temp_fx_thisFY.iloc[:, 3:]
fx_lastFY=temp_fx_lastFY.iloc[:, 3:]

fx_thisFY.set_index(['Currency(from)', 'Currency(to)'], inplace=True)
fx_lastFY.set_index(['Currency(from)', 'Currency(to)'], inplace=True)

fx_temp = pd.concat([fx_lastFY, fx_thisFY], axis=1).reset_index()
fx_temp.head(2)

,Currency(from),Currency(to),201604,201605,201606,201607,201608,201609,201610,201611,201612,201701,201702,201703,201704,201705,201706,201707,201708
0,AUD,AUD,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
1,AUD,JPY,84.27,79.58,78.2,78.16,77.22,77.38,79.11,81.58,85.22,85.75,86.62,86.12,83.08,83.44,83.74,87.53,87.06


In [22]:
# 計算に使用できるようレイアウトを変更
# Exchange Rate - change the layout
fx_rate = fx_temp[(fx_temp['Currency(from)']=='EUR') & (fx_temp['Currency(to)']=='JPY')].iloc[:, 2:].T
fx_rate.columns=['Fx Rate']
fx_rate['Currency']='EUR'
fx_rate['key']=1
fx_rate.reset_index(inplace=True)
fx_rate.columns=['Month', 'Fx Rate', 'Currency', 'key']
fx_rate['Month']=fx_rate['Month'].astype(int)

fx_rate.drop_duplicates(inplace=True)

fx_rate.tail()

,Month,Fx Rate,Currency,key
12,201704,117.95,EUR,1
13,201705,124.10,EUR,1
14,201706,124.52,EUR,1
15,201707,129.42,EUR,1
16,201708,129.88,EUR,1


In [23]:
# マージン率を読み込んで、計算に使用できるよう変更
# Margin Rate
margin['Partner2'] = margin['Customer Name'].str.replace('その他', 'Others')
#margin.set_index('Customer Name', inplace=True)
margin.drop_duplicates(inplace=True)

margin['key']=1
margin.head()

,Customer Name,マージン率/%,Partner2,key
0,その他,0.0,Others,1
1,TA Ceska,24.8,TA Ceska,1
2,TA Deutschland GmbH,27.6,TA Deutschland GmbH,1
3,TA Österreich GmbH,39.9,TA Österreich GmbH,1
4,TA Schweiz AG,45.8,TA Schweiz AG,1


In [24]:
# 月毎、TA会社毎の変換レート計算　変換レート＝為替/(1-マージン率/100)
rate = pd.merge(fx_rate, margin, on ='key')

rate = rate[['Month', 'Partner2', 'Currency', 'Fx Rate', 'マージン率/%']]
rate['Conversion Rate'] = rate['Fx Rate'] /(1-rate['マージン率/%']/100) 
rate['Month'] = rate['Month'].astype(int)

# 変換レート結果
# Necessary columns
rate2 = rate[['Month', 'Conversion Rate', 'Partner2']]
rate2.tail(5)

,Month,Conversion Rate,Partner2
80,201708,129.880000,Others
81,201708,172.712766,TA Ceska
82,201708,179.392265,TA Deutschland GmbH
83,201708,216.106489,TA Österreich GmbH
84,201708,239.630996,TA Schweiz AG


In [25]:
# TA変換レートをファイルに出力
output_rate = outputPath + r'TA_conversion_rate_{0}.xlsx'.format(str(report_date.date()))
writer = pd.ExcelWriter(output_rate)

rate.to_excel(writer, index=False)
writer.save()
#writer.close()

In [26]:
ta_data2.head(2)

,Month,customer no,Partner,Partner2,Group,description,QTY,total amount,Currency
0,201604,1101773280,Tier GmbH,Others,TA,Facial Massage Roller for Cats,1.0,47.93,Euro
1,201604,1101772220,TA Deutschland GmbH,TA Deutschland GmbH,TA,Facial Massage Roller for Cats,8.0,281.02,Euro


## Apply Conversion Rate to TA data  変換レートをTA販売実績に適用

In [27]:
# Join TA data and exhange rate TA販売データと変換レートをマージ
ta_data3 = pd.merge(ta_data2, rate2, how='left', on =['Partner2', 'Month'])
ta_data3.head(2)

,Month,customer no,Partner,Partner2,Group,description,QTY,total amount,Currency,Conversion Rate
0,201604,1101773280,Tier GmbH,Others,TA,Facial Massage Roller for Cats,1.0,47.93,Euro,124.620000
1,201604,1101772220,TA Deutschland GmbH,TA Deutschland GmbH,TA,Facial Massage Roller for Cats,8.0,281.02,Euro,172.127072


In [28]:
#　日本円に変換して'Sales JPY'カラムに追加
ta_data3['Sales JPY'] = round(ta_data3['total amount']* ta_data3['Conversion Rate'], 3)
#ta_data3.head(2)

In [29]:
# Check for records with Null ExRate　変換レートのない記録をチェック
ta_data3[ta_data3['Conversion Rate'].isnull()]['Partner2'].unique()

array([], dtype=object)

In [30]:
# Change TA data column name to match KDC column names
col_list = kdc_data2.columns.tolist()
ta_data3.columns = col_list

# Set '3rd Party or Inter'=3 for TA　'3rd Party or Inter'カラムは３とする
ta_data3['3rd Party or Inter'] = 3

ta_data3.head(2)

,Year / Month,Company Code,Company Name,Company Group Name,3rd Party or Inter,Material Name,Sales Quantity,Actual Sales,Plan Currency,ExRate,Actual Sales (Ex)
0,201604,1101773280,Tier GmbH,Others,3,Facial Massage Roller for Cats,1.0,47.93,Euro,124.620000,5973.037
1,201604,1101772220,TA Deutschland GmbH,TA Deutschland GmbH,3,Facial Massage Roller for Cats,8.0,281.02,Euro,172.127072,48371.150


In [31]:
# 記録数の確認
print('Number of record KDC', kdc_data2.shape)
print('Number of recor TA', ta_data3.shape)
print('Number of record KDC + TA:', len(kdc_data2) + len(ta_data3))

Number of record KDC (71664, 11)
Number of recor TA (15501, 11)
Number of record KDC + TA: 87165


## Merge KDC and TA data

In [32]:
# Add TA data to KDC data 　　KDCとTAを合わせる
data_merged = pd.concat([kdc_data2 ,ta_data3])

print(data_merged.shape)
# Add a Period column
data_merged['Year-Month'] = pd.to_datetime(data_merged['Year / Month'].astype(str)
                            .apply(lambda x: x[:4]+'-'+x[4:7])).dt.to_period('m')

# report_dateより後のデータは削除, 3rd party のみに
data_merged = data_merged[data_merged['Year-Month'] <= report_date.to_period('m')]
#data_merged = data_merged[data_merged['Year / Month'] <= report_date]
data_merged = data_merged[(data_merged['3rd Party or Inter']==3)]
print(data_merged.shape)

(87165, 11)
(86635, 12)


In [33]:
# Add display company name  販社名から数字、-Gなどを削除して'Display Co Name'カラムに。TA*販社はTAとする。
data_merged['Display Co Name'] = data_merged['Company Group Name']
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('[0-1][0-9].', '',  x))
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('OCEANIA-G', 'KDAU-G',  x))
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('TA \D*', 'TA', x))
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('Others', 'TA',  x))
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('-G', '',  x))

print('Company Names: ',  data_merged['Display Co Name'].unique())

Company Names:  [' Pow Japan' ' Pow Europe' ' Pow America' ' Pow Oceania' ' Pow Asia'
 ' Pow Canada' 'TA']


In [34]:
#pickle_dir='/Users/yuka/Documents/Python/Kyocera/OptionSales/'
pickle_dir = 'C:\\Users\\ymatsukawa\\AppData\\home\\ymatsukawa\\'
data_merged.to_pickle(pickle_dir+'options sales report merged data.pkl')

In [35]:
#data_merged = pd.read_pickle(pickle_dir + 'options sales report merged data.pkl')

In [36]:
# Crate 'Month-Year' column (date period(M))
#data_merged['Month']=data_merged['Year / Month'].astype(str)
#data_merged['Month']=data_merged['Month'].apply(lambda x: x[:4]+'-'+x[4:7]+'-'+'01').astype('datetime64').dt.strftime('%Y-%m')

#data_merged['Year-Month'] = pd.to_datetime(data_merged['Year / Month'].astype(str)
#                            .apply(lambda x: x[:4]+'-'+x[4:7])).dt.to_period('m')
data_merged.head(2)

,Year / Month,Company Code,Company Name,Company Group Name,3rd Party or Inter,Material Name,Sales Quantity,Actual Sales,Plan Currency,ExRate,Actual Sales (Ex),Year-Month,Display Co Name
7768,201604,1101,Amazin' Animals Pet Shop,09. Pow Japan,3,Non target products,4.0,21000.0,JPY,1.0,21000.0,2016-04,Pow Japan
7769,201604,1101,Amazin' Animals Pet Shop,09. Pow Japan,3,Non target products,4.0,5460.0,JPY,1.0,5460.0,2016-04,Pow Japan


In [37]:
# Apply the function above 'Extracted Name'カラムに関数を適用
data_merged['Extracted Name'] = data_merged['Material Name']

In [38]:
# 集計対象商材のグループ定義
mapping = [ ('Wobble Wag Giggle Ball', ['Wobble Wag Giggle Ball', 'Pet Zone IQ Treat Ball']),
            ('Pet Bed & Comfort', ['Sneaker Pet Bed','Catit Design Senses Massage Center', 'Suck UK Laptop Cat Scratching Pad', 'K&H Thermo Cat Windowsill Seat', 'KritterWorld Microplush Pet Blanket']),
            ('Holiday & Subscription Box', ['PupBox Holiday Box', 'Barkbox Subscription Box', 'MeowBox Monthly Subscription Box']),
            ('Furbo Dog Camera', ['Furbo Dog Camera', 'Petcube Camera'])
           ] 

d = dict()
for (value, keys) in mapping:
    for key in keys:
        d[key] = value
        

#  '対象商材'カラム追加して上記のグループを適用 - just rename
data_merged['対象商材'] = data_merged['Extracted Name'].apply(lambda x: d[x] if x in d else x)

In [39]:
#　'Material Name'から'対象商材'のマッピングをファイルに出力
name_col = ['対象商材', 'Extracted Name','Material Name']
product_mapping = data_merged[data_merged['対象商材'].notnull()][name_col].groupby(by=name_col).count() 


outfile_product_mapping = outputPath + '対象商材マッピング_{0}.xlsx'.format(str(report_date.date()))
writer = pd.ExcelWriter(outfile_product_mapping)

product_mapping.to_excel(writer, index=True)
writer.save()
writer.close()

In [40]:
# 抽出したいアイテムのロジックに当てはまらなかった Material List 商材をファイルに出力
product_not_mapped = data_merged[data_merged['対象商材'].isnull()][name_col].groupby(by='Material Name').count() 
#product_not_mapped.sort_values(by='Material Name')


outfile_product_mapping = outputPath + 'マッピングされなかった商材_{0}.xlsx'.format(str(report_date.date()))

writer = pd.ExcelWriter(outfile_product_mapping)
product_not_mapped.to_excel(writer, index=True)

writer.save()
writer.close()

In [41]:
# 集計に必要なデータに限定する


# Limit to reporting items レポート対象商材に限定
report_items = data_merged[data_merged['対象商材'].notnull()]['対象商材'].unique().tolist()
report_items.remove('Non target products')
report_items=sorted(report_items)
#print('レポート対象商材\n', report_items)


# Limit to reporting proucts and the last x years 過去x年に限定
report_start_yr =current_FY_start - relativedelta(years=num_yr_to_aggregate)

#standardize_sales_data = data_merged[(data_merged['対象商材'].notnull()) & (data_merged['Year / Month']>=report_start_yr)].copy() 
standardize_sales_data = data_merged[(data_merged['対象商材'].isin(report_items)) 
                                     & (data_merged['Year-Month']>=report_start_yr.to_period('m'))].copy() 


# ソート
standardize_sales_data.sort_values(by=['対象商材', 'Extracted Name', 'Year-Month', 'Display Co Name'], inplace=True)

#カラム順序並び替え
standardize_sales_data=standardize_sales_data[['Year-Month', 'Company Code', 'Company Name', 'Company Group Name', 
                                               'Display Co Name','3rd Party or Inter', 
                                               'Material Name', 'Extracted Name', '対象商材', 'Sales Quantity', 
                                               'Actual Sales', 'Plan Currency', 'ExRate', 'Actual Sales (Ex)']]

standardize_sales_data.head(2)

,Year-Month,Company Code,Company Name,Company Group Name,Display Co Name,3rd Party or Inter,Material Name,Extracted Name,対象商材,Sales Quantity,Actual Sales,Plan Currency,ExRate,Actual Sales (Ex)
52363,2016-04,1201,Pets Unleashed,04. Pow America,Pow America,3,Catit Senses 2.0 Circuit,Catit Senses 2.0 Circuit,Catit Senses 2.0 Circuit,4.0,540.0,USD,109.93,59362.2
58373,2016-04,1202,Pets Unlimited,04. Pow America,Pow America,3,Catit Senses 2.0 Circuit,Catit Senses 2.0 Circuit,Catit Senses 2.0 Circuit,2.0,0.0,CAD,NaN,0.0


## Write merged sales data to output (BIツール用　統一したKDCとTA販売データを出力)

In [42]:
## BIツール用　集計してないデータを出力　TAマージン率と為替変換は済み

In [43]:
output_data = outputPath + 'BIツール用集計してない実績_{0}.xlsx'.format(str(report_date.date()))
writer = pd.ExcelWriter(output_data)

standardize_sales_data.to_excel(writer, 'Sales Data', index=False)
writer.save()
#writer.close()

## Write aggregated data to output (集計結果の出力)

In [44]:
display_cols = ['Year-Month', '3rd Party or Inter', 'Display Co Name', '対象商材', 'Sales Quantity', 'Actual Sales (Ex)']
agg_cols = ['対象商材', '3rd Party or Inter' ,'Year-Month', 'Display Co Name' ]

agg_sales_last5FY = standardize_sales_data[display_cols].groupby(by=agg_cols).sum()
agg_sales_last5FY['Actual Sales (Ex) in 1000']  = agg_sales_last5FY['Actual Sales (Ex)']/1000

agg_sales_last5FY.head()

Sales Quantity  \
対象商材                     3rd Party or Inter Year-Month Display Co Name                   
Catit Senses 2.0 Circuit 3                  2016-04     Pow America                6.0   
                                                        Pow Europe               384.0   
                                                        Pow Japan                 12.0   
                                                        Pow Oceania                1.0   
                                                       TA                          3.0   

                                                                        Actual Sales (Ex)  \
対象商材                     3rd Party or Inter Year-Month Display Co Name                      
Catit Senses 2.0 Circuit 3                  2016-04     Pow America             59362.200   
                                                        Pow Europe            1873872.290   
                                                        Pow Japan              183000.000   
                                                        Pow Oceania              6151.710   
                                                       TA                       24970.715   

                                                                        Actual Sales (Ex) in 1000  
対象商材                     3rd Party or Inter Year-Month Display Co Name                             
Catit Senses 2.0 Circuit 3                  2016-04     Pow America                     59.362200  
                                                        Pow Europe                    1873.872290  
                                                        Pow Japan                      183.000000  
                                                        Pow Oceania                      6.151710  
                                                       TA                               24.970715

In [45]:
# t=agg_sales_last5FY.reset_index()
# t.head()

In [46]:
output_agg = outputPath + '集計結果_Last5FY_{0}.xlsx'.format(str(report_date.date()))
#output_agg = r'C:\Users\ymatsukawa\Documents\Project\OptionSales\集計結果_Last5FY_{0}_new3.xlsx'.format(str(report_date.date()))
writer = pd.ExcelWriter(output_agg)

agg_sales_last5FY.reset_index().to_excel(writer, 'Aggregated', index=False)
writer.save()

## Plot Graphs

In [47]:
# # グラフ配色 RGB
# # Tableau colors - These are the "Tableau 20" colors as RGB.    
# tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
#              (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
#              (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
#              (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
#              (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
  
# # Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
# for i in range(len(tableau20)):    
#     r, g, b = tableau20[i]    
#     tableau20[i] = (r / 255., g / 255., b / 255.)    

# #colors = ['peru', 'gold', 'olivedrab', 'orange', 'steelblue', 'lightcoral', 'darkslateblue', 'tan', 'palevioletred']

In [48]:
# # 関数　右Y軸と左Y軸の０を同じ高さに揃える
# def align_yaxis(ax1, ax2):
#     """Align zeros of the two axes, zooming them out by same ratio"""
#     axes = (ax1, ax2)
#     extrema = [ax.get_ylim() for ax in axes]
#     tops = [extr[1] / (extr[1] - extr[0]) for extr in extrema]
#     # Ensure that plots (intervals) are ordered bottom to top:
#     if tops[0] > tops[1]:
#         axes, extrema, tops = [list(reversed(l)) for l in (axes, extrema, tops)]

#     # How much would the plot overflow if we kept current zoom levels?
#     tot_span = tops[1] + 1 - tops[0]

#     b_new_t = extrema[0][0] + tot_span * (extrema[0][1] - extrema[0][0])
#     t_new_b = extrema[1][1] - tot_span * (extrema[1][1] - extrema[1][0])
#     axes[0].set_ylim(extrema[0][0], b_new_t)
#     axes[1].set_ylim(t_new_b, extrema[1][1])

In [49]:
# # Functions used for charts
# # These need to be defined in the box below
# #
# # current_FY_start_mo = current_FY_start.to_period('m')
# # last_FY_start_mo = last_FY_start.to_period('m')
# # report_date_mo = report_date.to_period('m')

# # Insert 0 if there is no entry for the month
# def getChartData(df, item):
# #agg_sales_last5FY.iloc[agg_sales_last5FY.index.get_level_values('対象商材') == item].reset_index()
#     df1 = df[(df['対象商材']==item) & (df['Year-Month']>=last_FY_start_mo)]
    
    
#     # create df with months of chart range
#     idx = pd.date_range(last_FY_start, report_date, freq='M').to_period('m')
#     d={'Year-Month':list(idx), 'key':1}
#     df_month=pd.DataFrame(d)

#     # create df with company that has a record in this FY and last FY
#     co = df1['Display Co Name'].unique().tolist()
#     d = {'Display Co Name': co, 'key':1}
#     df_co = pd.DataFrame(d)
    
#     df_month_co = pd.merge(df_month, df_co, how='inner', on='key')
    
#     df2=pd.merge(df_month_co, df1, how ='left', on = ['Year-Month', 'Display Co Name'])
#     df2.fillna(value =0, inplace=True)
#     df2['対象商材']=item
#     df2['3rd Party or Inter']=3
 
#     return df2


# def get_thisFY_pivot_df(df): 
    
#     df = df[(df['Year-Month']>= current_FY_start_mo) & (df['Year-Month']<= report_date_mo)].pivot(index='Year-Month', 
#                                                                                                   columns='Display Co Name')['Actual Sales (Ex) in 1000']
    
#     return df



# def get_chart_df(df, start_mo, end_mo):
    
#     df_FY = df[(df['Year-Month']>= start_mo) & (df['Year-Month']<= end_mo)].copy()
#     df_FY_agg = df_FY.groupby(by='Year-Month').sum().reset_index()[['Year-Month', 'Actual Sales (Ex) in 1000']]

#     # Add cumulative column
#     df_FY_agg['Cumulative'] = df_FY_agg['Actual Sales (Ex) in 1000'].cumsum()

#     return df_FY_agg

In [50]:
# # Function to plot chart
# def plotChart(df_thisFY_pivot, df_thisFY, df_lastFY):
    
    
#      # 棒グラフ配色
#     colors = tableau20
    
#     width = 0.3
#     shift = width/2

#     fig = plt.figure(figsize=(12, 6))
#     ax = plt.subplot(111)

#     #　今年度棒グラフ
#     df_thisFY_pivot.plot(kind='bar', stacked=True, width=width, ax=ax, position = 0, 
#                          color = colors if df_thisFY_pivot.shape[1]>1 else colors[0], linewidth = 0.5)
 
#     # 前年度棒グラフ
#     df_lastFY['Actual Sales (Ex) in 1000'].plot(kind='bar', width=width, ax=ax, color='darkgray', label='Last Year', 
#                                                 position=1, linewidth = 0, grid=True)
#     #ax2 = df_lastFY['Actual Sales (Ex) in 1000'].plot(kind='bar', width=width, ax=ax, color='darkgray', label='Last Year', position=1, linewidth = 0, grid=True)

#     # 前年度折れ線グラフ
#     ax2=df_lastFY['Cumulative'].plot(kind='line', color=tableau20[6], secondary_y=True, linewidth=2, style='.--' ,
#                                              grid=True, label='Cumulative Last Year')

#     # 今年度折れ線グラフ
#     ax2=df_thisFY['Cumulative'].plot(kind='line', color=tableau20[6], secondary_y=True, linewidth=2, style='.-' ,
#                                              grid=True, label='Cumulative This Year')


#     #fig.suptitle('Super title')
#     plt.title('FY2018 {0} Sales Result (3rd Party Sales)'.format(item), fontsize=14)

#     ax.set_xlabel(' ', fontsize = 10) #Month
#     ax.set_ylabel('Unit in 1,000 Yen', fontsize = 12)
#     ax2.set_ylabel('Cumulative', fontsize = 12)

#     plt.xlim([-1,12])
#     plt.xticks(range(12), ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar'], rotation=360)

#     # Legend
#     ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), shadow=False, ncol=5, frameon=False) # under
#     ax2.legend(loc='center', bbox_to_anchor=(0.5, -0.08), shadow=False, ncol=5, frameon=False) # under

    
#     # Value label 数値ラベル

#     # 今年度折れ線グラフ
#     for i, label in enumerate(list(df_thisFY.index)):
#         height = df_thisFY.ix[label]['Cumulative']
#         #offset = height*.10 if height < 500 else height/(np.power(np.log(height), 2)*2)
#         offset = 0
#         ax2.annotate('¥{0:,.{1}f}'.format(height, 0), (i, height), clip_on=True, color='navy', fontsize=11,
#                      weight = 'bold', va='bottom', horizontalalignment = 'right') # height+offset

#     # 昨年度折れ線グラフ
#     for i, label in enumerate(list(df_lastFY.index)):
#         height = df_lastFY.ix[label]['Cumulative'] 
#         #offset = height*.10 if height < 500 else height/(np.power(np.log(height), 2)*2)
#         offset = 1 if height >= 20 else -1
#         ax2.annotate('¥{0:,.{1}f}'.format(height, 0), (i, height-offset), clip_on=True, color=tableau20[6], fontsize=11,
#                     va='top' if height >= 20 else 'bottom', 
#                     horizontalalignment = 'left')

    
#     align_yaxis(ax, ax2)
    
    
#      # pngに出力
#     output_chart = outputPath + '{0}_{1}.png'.format(item, report_date.date())
#     plt.savefig(output_chart, dpi=300, bbox_inches='tight')
#     #plt.savefig(output_chart, dpi=300)

#     plt.close(fig)

In [51]:
# Create Last FY and This FY's data df and plot charts, save to files

# width = 0.3
# shift = width/2

# Convert date to 'Month'-period
report_date_mo = report_date.to_period('m')
current_FY_start_mo = current_FY_start.to_period('m')
last_FY_start_mo = last_FY_start.to_period('m')
last_FY_end_mo = (current_FY_start-relativedelta(months=1)).to_period('m')


reset_data = agg_sales_last5FY.reset_index()

#report_items=['CloudDirect']
for item in report_items:
    print('Creating a graph for', item)
    
    temp2 = utils.getChartData(reset_data, item, last_FY_start_mo, last_FY_start, report_date)
    #print(temp2)
    
    # This FY data, pivotted (multiple columns for company) for stacked bar graph
    df_thisFY_pivot = utils.get_thisFY_pivot_df(temp2, current_FY_start_mo, report_date_mo)
    #df_thisFY_pivot =temp2[(temp2['Year-Month']>= current_FY_start_mo) & (temp2['Year-Month']<= report_date_mo)].pivot(index='Year-Month', columns='Display Co Name')['Actual Sales (Ex) in 1000']

    # This FY df, NOT pivotted for this FY line graph
    chart_df_thisFY = utils.get_chart_df(temp2, current_FY_start_mo, report_date_mo)

    # Last FY df, NOT pivotted for both bar and line graph
    chart_df_lastFY = utils.get_chart_df(temp2, last_FY_start_mo, last_FY_end_mo)
    
    # Plot chart
    utils.plotChart(df_thisFY_pivot, chart_df_thisFY, chart_df_lastFY, item, report_date, outputPath)


Creating a graph for Catit Senses 2.0 Circuit
Creating a graph for Doggie Drinklets Portable Water Bowl
Creating a graph for Drs. Foster & Smith Whack-a-Mouse Cat Toy
Creating a graph for Facial Massage Roller for Cats
Creating a graph for FireboltCreations Custom Pet Pillow
Creating a graph for Furbo Dog Camera
Creating a graph for H2O4K9 Doggy Water Bottle
Creating a graph for Hammacher Schlemmer Feline's Laser Chasing Scratch Post
Creating a graph for Holiday & Subscription Box
Creating a graph for Kong Rubber Dog Frisbee
Creating a graph for Magic Latch Leash Connection Sytem
Creating a graph for Modkat Litter Box
Creating a graph for Munchiecat Sushi Cat Toys
Creating a graph for Pet Bed & Comfort
Creating a graph for Pet Gear Pup Tub
Creating a graph for Petcube Play
Creating a graph for Pooch Selfie: The Original Dog Selfie Stick
Creating a graph for Siensync Interactive Feeder Dog Bowl
Creating a graph for Sneaker Pet Bed 
Creating a graph for The Honest Kitchen Bone Broth
Crea